In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import sys
import matplotlib.pyplot as plt

sys.path.append('../../src/')
import cross_validation as cv
from plotting import ws_style
plt.style.use(ws_style)

In [ ]:
parameter_names = ['iterations'	,'shots', 'rounds']
# split indices go from 1 to 10
curpath = os.path.abspath('.')
folders = [os.path.join(curpath, 'runs', 'run{}'.format(split),'checkpoints') for split in range(1,11)]
list_of_expts = ['Projection from TrainingStats', 'Projection from TrainingResults']
colors_dict = {'baseline' : 'k',
               'Projection from TrainingStats' : 'tab:blue',
               'Projection from TrainingResults' : 'tab:orange',
               'Random Exploration' : 'green',
               'SequentialSearch: Id=cold' : 'red',
               'SequentialSearch: Id=warm' : 'purple'
               }
response_string = 'Approximation Ratio\n' + r"r=$\frac{\langle \vec{\beta}, \vec{\gamma} | H | \vec{\beta}, \vec{\gamma} \rangle}{E_{max}}$" # Used for y-label on plots 
resource_string = r'Resource = Restarts $\times$ shots $\times$ COBLYA Iterations' # Used for x-label on plots 

In [ ]:
# Load data for parameter plotting
cv.load_parameters(folders, list_of_expts)
cv.process_params_across_splits(parameter_names)

# Load data for performance plotting
cv.load_performance(folders, list_of_expts)
cv.process_performance_across_splits(parameter_names)
# pd.read_csv(os.path.join(folders[0], 'params_plotting', 'baseline.csv'))

## Plot Parameters

In [ ]:
for param in parameter_names:
    fig, axs = plt.subplots()
    for expt in cv.parameters_summarized_dict.keys():
        expt_param_df = cv.parameters_summarized_dict[expt][param]
        y = expt_param_df['mean']
        x = expt_param_df['resource']
        y_u = expt_param_df['CI_u']
        y_l = expt_param_df['CI_l']
        if expt == 'baseline':
            lw = 1
            alpha = 0.1
            label = 'VirtualBest'
        else:
            lw = 1.5
            alpha = 0.25
            label = expt
        
        axs.fill_between(x, y_l, y_u ,alpha=alpha, lw=0, color=colors_dict[expt])
        axs.plot(x, y , '-o', lw=lw, color=colors_dict[expt], label=label)
    axs.set_xscale('log')
    axs.set_xlabel(resource_string)
    axs.set_ylabel(param)
    axs.grid(axis="y")
    axs.set_axisbelow(True)
    axs.legend(loc='best')
    fig.tight_layout()
    
    figsaveloc = os.path.join('figures','all_splits')
    if not os.path.exists(figsaveloc): os.makedirs(figsaveloc)
    figname = os.path.join(figsaveloc, param)
    fig.savefig(figname+'.png', dpi=300)
    fig.savefig(figname+'.pdf')


## Plot Performance

In [ ]:
fig, axs = plt.subplots()

for expt, expt_df in cv.performance_summarized_dict.items():
    x = expt_df['resource']
    y = expt_df['mean']
    y_u = expt_df['CI_u']
    y_l = expt_df['CI_l']
    if expt == 'baseline':
        lw = 1
        alpha = 0.1
        label = 'VirtualBest'
    else:
        lw = 1.5
        alpha = 0.25
        label = expt
    axs.fill_between(x, y_l, y_u, alpha=alpha, lw=0, color=colors_dict[expt])
    axs.plot(x, y, '-o', lw=lw, color=colors_dict[expt], label=label)
axs.set_xscale('log')
axs.set_xlabel(resource_string)
axs.set_ylabel(response_string)
axs.grid(axis="y")
axs.set_axisbelow(True)
axs.legend(loc='best')
fig.tight_layout()

figsaveloc = os.path.join('figures','all_splits')
if not os.path.exists(figsaveloc): os.makedirs(figsaveloc)
figname = os.path.join(figsaveloc, 'performance')
fig.savefig(figname+'.png', dpi=300)
fig.savefig(figname+'.pdf')

## Individual splits

In [ ]:
split_ind=0

In [ ]:
# Plot parameters
for param in parameter_names:
    fig, axs = plt.subplots()
    for expt, expt_df in cv.parameters_dict.items():
        expt_split_df = expt_df[expt_df['split_ind'] == split_ind]
        x = expt_split_df['resource']
        y = expt_split_df[param]
        label = expt
        if expt == 'baseline':
            lw = 1
            label = 'VirtualBest'
        else:
            lw = 1.5
        axs.plot(x, y , '-o', lw=lw, color=colors_dict[expt], label=label)
    
    axs.set_xscale('log')
    axs.set_xlabel(resource_string)
    axs.set_ylabel(param)
    axs.grid(axis="y")
    axs.set_axisbelow(True)
    axs.legend(loc='best')
    fig.tight_layout()
    
    figsaveloc = os.path.join('figures','split_ind={}'.format(split_ind))
    if not os.path.exists(figsaveloc): os.makedirs(figsaveloc)
    figname = os.path.join(figsaveloc, param)
    fig.savefig(figname+'.png', dpi=300)
    fig.savefig(figname+'.pdf')

In [ ]:
# Plot performance
fig, axs = plt.subplots()

for expt, expt_df in cv.performance_dict.items():
    expt_split_df = expt_df[expt_df['split_ind'] == split_ind]
    y = expt_split_df['response']
    x = expt_split_df['resource']
    label = expt
    if expt == 'baseline':
        lw = 1
        alpha = 0.1
        label = 'VirtualBest'
    else:
        lw = 1.5
        alpha = 0.25
    
    axs.plot(x, y , '-o', lw=lw, color=colors_dict[expt], label=label)
    if 'response_lower' in expt_split_df.columns:
        # Add CI, if they exist in the data
        y_u = expt_split_df['response_upper']
        y_l = expt_split_df['response_lower']
        axs.fill_between(x, y_l, y_u ,alpha=alpha, lw=0, color=colors_dict[expt])
        
    
axs.set_xscale('log')
axs.set_xlabel(resource_string)
axs.set_ylabel(response_string)
axs.grid(axis="y")
axs.set_axisbelow(True)
axs.legend(loc='best')
fig.tight_layout()

figsaveloc = os.path.join('figures','split_ind={}'.format(split_ind))
if not os.path.exists(figsaveloc): os.makedirs(figsaveloc)
figname = os.path.join(figsaveloc, 'performance')
fig.savefig(figname+'.png', dpi=300)
fig.savefig(figname+'.pdf')